In [1]:
import numpy as np
import pandas as pd 
import cv2
import numpy as np

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/traindata/train_35.jpg
/kaggle/input/traindata/train_18.jpg
/kaggle/input/traindata/train_58.jpg
/kaggle/input/traindata/train_49.jpg
/kaggle/input/traindata/train_42.jpg
/kaggle/input/traindata/train_68.jpg
/kaggle/input/traindata/train_27.jpg
/kaggle/input/traindata/train_20.jpg
/kaggle/input/traindata/train_77.jpg
/kaggle/input/traindata/train_41.jpg
/kaggle/input/traindata/train_50.jpg
/kaggle/input/traindata/train_70.jpg
/kaggle/input/traindata/train_28.jpg
/kaggle/input/traindata/train_10.jpg
/kaggle/input/traindata/train_15.jpg
/kaggle/input/traindata/train_17.jpg
/kaggle/input/traindata/train_34.jpg
/kaggle/input/traindata/train_64.jpg
/kaggle/input/traindata/train_11.jpg
/kaggle/input/traindata/train_56.jpg
/kaggle/input/traindata/train_65.jpg
/kaggle/input/traindata/train_39.jpg
/kaggle/input/traindata/train_46.jpg
/kaggle/input/traindata/train_33.jpg
/kaggle/input/traindata/train_26.jpg
/kaggle/input/traindata/train_30.jpg
/kaggle/input/traindata/train_19.jpg
/

In [2]:
#Ana Maria C. Wagner
#PDI 
#2023.2

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Define a function to split the image into three ps
def image_split(img, y_start, plot=False):

    # Convert image to RGB format
    image = img
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert RGB image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Get the height and width of the image
    height, width = image.shape

    # Calculate split points for width and height
    pointSplit1_width = width // 3
    pointSplit2_width = 2 * width // 3
    pointSplit1_height = height // 2

    # Split the image into three ps
    p1 = image[y_start:pointSplit1_height, :pointSplit1_width]
    p2 = image[y_start:pointSplit1_height, pointSplit1_width:pointSplit2_width]
    p3 = image[y_start:pointSplit1_height, pointSplit2_width:]
    
        # Plot the image ps if plot is True
    if plot:
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(p1, cv2.COLOR_BGR2RGB))
        plt.title('p 1')
    
        plt.subplot(1, 3, 2)
        plt.imshow(cv2.cvtColor(p2, cv2.COLOR_BGR2RGB))
        plt.title('p 2')
    
        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(p3, cv2.COLOR_BGR2RGB))
        plt.title('p 3')
        
        plt.show()

    return p1, p2, p3


In [5]:
# Define a function to calculate the mean of a region of interest (ROI)
def MeanOfRoi(img, row_start, row_end, col_start, col_end, target, is_high, print_result=False):
    # Get the ROI
    roi = img[row_start:row_end, col_start:col_end]
    # Calculate the mean intensity of the ROI
    mean = np.mean(roi)
    
    # Print the result if print_result is True
    if print_result:
        print("Mean value based on ROI:", mean)
 
    # Check if the mean intensity satisfies the condition based on is_high
    if is_high:
        if mean < target:
            return 1
        return 0
    else:
        if mean > target:
            return 1
        return 0

In [6]:
# Define a function for image Segmt
def Segmt(img, plot=False):
    # Threshold the image
    _, image = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Plot the segmented image if plot is True
    if plot:
        plt.imshow(image, cmap='gray')
        plt.show()
        
    return image


In [7]:
# Function to check for high content
def HC_check(img):
    res = []
    imgs = image_split(img, y_start=60)

    for img_piece in imgs:
        segmented_img = Segmt(img_piece)
        res.append(segmented_img)

    return MeanOfRoi(res[1], 60, 75, 40, 60, target=60, is_high=True)

In [8]:
# Function to check for low content
def LC_check(img):
    imgs = image_split(img, y_start=60)
    segmented_img = Segmt(imgs[1])

    return MeanOfRoi(segmented_img, 80, 84, 75, 80, target=240, is_high=False)

In [9]:
# Function to check for missing cover
def cover_check(img):
    img_ps = image_split(img, y_start=0, plot=False)
    img_segmented = Segmt(img_ps[1], plot=False)

    res = MeanOfRoi(img_segmented, 15, 30, 60, 75, target=240, is_high=False)

    return res

# Function to check for white label
def WL_check(img):
    res = Segmt(img, plot=False)
    return MeanOfRoi(res, 200, 280, 150, 200, is_high=False, target=230)

In [10]:

# Function to check for missing label
def check_label_none(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = Segmt(img_gray, plot=False)
    return MeanOfRoi(res, 200, 280, 150, 200, is_high=True, target=10)


In [11]:
###### 
# Function to check for missing bottle
def check_bottle_none(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = Segmt(gray, plot=False)
    return MeanOfRoi(res, 160, 180, 150, 200, target=250, is_high=False)


In [12]:
# Function to check for smashed bottle
def check_bottle_smashed(img):
    def get_contour(img, left=True):
        if left:
            return img[10:210, 125:130]
        else:
            return img[120:200, 245:250]

    def verify(img):
        img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        threshold = 100
        _, img_bin = cv2.threshold(img_bw, threshold, 255, cv2.THRESH_BINARY)
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower_red = np.array([0, 100, 100])  
        upper_red = np.array([10, 255, 255])  

        mask_red = cv2.inRange(img_hsv, lower_red, upper_red)
        img_bin[mask_red == 255] = 0
        img_bin[img_bw == 0] = 0

        return 1 if not np.any(img_bin == 0) else 0

    left_bottle = get_contour(img, left=True)
    right_bottle = get_contour(img, left=False)

    return verify(left_bottle) or verify(right_bottle)


In [13]:

# Function to check for misplaced label
def check_label_misplaced(img):
    if check_label_none(img) == 1:
        return 0
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    y1, y2 = 170, 210
    x1, x2 = 100, 225
    cropped = img[y1:y2, x1:x2]
    
    mean_intensity = cv2.mean(cropped)[0]

    return 1 if mean_intensity < 80 else 0

In [14]:
########## # Function to detect bottle defects
def detect_bottles_defects(bottle, df, id):
    cols = ["id", "CONTENT_HIGH", "CONTENT_LOW", "COVER_NONE", "BOTTLE_SMASHED", "LABEL_WHITE", "LABEL_MISPLACED", "LABEL_NONE", "BOTTLE_NONE"]
    row = {"id": id}
    defects = [0, 0, 0, 0, 0, 0, 0, 0]

    file_number = int(id.split("_")[1].split(".")[0])
    row["file_number"] = file_number

    if check_bottle_none(bottle) == 1:
        row["BOTTLE_NONE"] = 1
        defects[7] = 1

    if check_label_none(bottle) == 1:
        row["LABEL_NONE"] = 1
        defects[6] = 1

    for col in cols[1:]:
        if not "BOTTLE_NONE" in row:
            if col == "CONTENT_HIGH":
                if HC_check(bottle) == 1:
                    row[col] = 1
                    defects[0] = 1
            elif col == "CONTENT_LOW":
                if LC_check(bottle) == 1:
                    row[col] = 1
                    defects[1] = 1
            elif col == "COVER_NONE":
                if cover_check(bottle) == 1:
                    row[col] = 1
                    defects[2] = 1
            elif col == "BOTTLE_SMASHED":
                if check_bottle_smashed(bottle) == 1:
                    row[col] = 1
                    defects[3] = 1
            elif col == "LABEL_WHITE" and not "LABEL_NONE" in row:
                if WL_check(bottle) == 1:
                    row[col] = 1
                    defects[4] = 1
            elif col == "LABEL_MISPLACED" and not "LABEL_NONE" in row and not "LABEL_WHITE" in row:
                if check_label_misplaced(bottle) == 1:
                    row[col] = 1
                    defects[5] = 1

    new_row_df = pd.DataFrame([row])

    df = pd.concat([df, new_row_df], ignore_index=True)
    
    return np.array(defects), df



In [15]:
def main():
    columns = ["filename", "CONTENT_HIGH", "CONTENT_LOW", "COVER_NONE", "BOTTLE_SMASHED", "LABEL_WHITE", "LABEL_MISPLACED", "LABEL_NONE", "BOTTLE_NONE"]
    df = pd.DataFrame(columns=columns)
    
    bottles_defects = []
    
    for i in range(1, 78):
        filename = f"train_{i}.jpg"
        bottle = cv2.imread(f"/kaggle/input/traindata/train_{i}.jpg")
        defects, df = detect_bottles_defects(bottle, df, filename)
        bottles_defects.append(defects)
        
    df.to_csv("sample_submission.csv", index=False)
    return bottles_defects

def print_defects_with_numbers(bottles_defects):
    for i, defects in enumerate(bottles_defects, start=1):
        print(f"Image 'test_{i}': {defects}")

bottles_defects = main()
print_defects_with_numbers(bottles_defects)

main()


Image 'test_1': [0 0 0 0 0 0 0 0]
Image 'test_2': [0 0 0 0 0 0 0 0]
Image 'test_3': [1 0 1 0 0 1 0 0]
Image 'test_4': [1 0 1 0 0 0 0 0]
Image 'test_5': [1 0 1 0 0 0 0 0]
Image 'test_6': [1 0 1 0 0 0 0 0]
Image 'test_7': [0 0 0 0 0 0 0 0]
Image 'test_8': [0 0 0 0 0 0 0 0]
Image 'test_9': [0 0 0 0 0 0 0 1]
Image 'test_10': [1 0 0 0 0 0 0 0]
Image 'test_11': [0 0 0 0 0 0 0 0]
Image 'test_12': [1 0 1 0 0 0 0 0]
Image 'test_13': [0 0 0 0 0 0 0 1]
Image 'test_14': [0 0 0 0 0 0 1 0]
Image 'test_15': [0 0 0 0 0 0 0 0]
Image 'test_16': [0 0 0 0 1 0 0 0]
Image 'test_17': [0 0 0 0 0 0 0 1]
Image 'test_18': [1 0 0 1 0 0 0 0]
Image 'test_19': [1 0 0 0 0 1 0 0]
Image 'test_20': [0 0 0 0 0 0 0 0]
Image 'test_21': [1 0 0 1 0 0 0 0]
Image 'test_22': [0 1 0 0 0 0 0 0]
Image 'test_23': [1 0 1 0 0 0 1 0]
Image 'test_24': [1 0 0 0 0 0 0 0]
Image 'test_25': [0 0 0 0 0 0 0 0]
Image 'test_26': [0 1 0 0 0 0 0 0]
Image 'test_27': [1 0 1 0 0 0 0 0]
Image 'test_28': [0 0 0 0 0 0 0 0]
Image 'test_29': [0 0 0 0 0 0

[array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 1, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 1, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 1, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 

In [16]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("sample_submission.csv")

# Display the DataFrame as a table
df.head(48)


,filename,CONTENT_HIGH,CONTENT_LOW,COVER_NONE,BOTTLE_SMASHED,LABEL_WHITE,LABEL_MISPLACED,LABEL_NONE,BOTTLE_NONE,id,file_number
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_1.jpg,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_2.jpg,2.0
2,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,train_3.jpg,3.0
3,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,train_4.jpg,4.0
4,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,train_5.jpg,5.0
5,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,train_6.jpg,6.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_7.jpg,7.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_8.jpg,8.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,train_9.jpg,9.0
9,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_10.jpg,10.0
